<center>
    <img src="./images/msp_header.png">
</center>

#### Prof. Dr. -Ing. Gerald Schuller <br> Jupyter Notebook: Renato Profeta

# Polyphase Representation

In [1]:
%%html
<center>
<iframe width="560" height="315" src="https://www.youtube.com/embed/oGT5GcOIvFo" frameborder="0" allow="accelerometer; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
</center>

Last time we saw how to obtain the polyphase representation for the fitering and downsampling operation of **1 fiter**. We now extend this formulation for a **bank of N fiters**.

Here we not only have 1 fiter, but N fiters in our **analysis fiter bank**, and hence can assemble the N fiter outputs or N subbands into a vector of N elements, 

$$\large
Y(z):=[Y_0(z),Y_1(z),...,Y_\text{N-1}(z)]$$

We also now have **N fiters** instead of 1 fiter, and we assemble the polyphase vectors of our fiters into a matrix, in which each column corresponds to an anlalysis filter, 

$$\large
H(z):=[H_0^T(z),H_1^T(z),...,H_\text{N-1}^T(z)]=$$

$$\large
=\left[ 
        \begin{array}{ccc} 
            H_{N-1,0}(z) & H_{N-1,1}(z) & \dots & H_{N-1,N-1}(z) \\ 
            H_{N-2,0}(z) & \dots & \dots & \vdots \\
            \vdots & \dots & \ddots & \vdots \\ 
            H_{0,0}(z) & \dots & \dots & H_{0,N-1}(z) 
        \end{array}
    \right]
$$,
with
$$\large
H_\text{n,k}(z)=\sum_{m=0}^{\infty}(m \cdot N+n) \cdot z^{-m}$$

In [2]:
%%html
<center>
<iframe width="560" height="315" src="https://www.youtube.com/embed/pOYWmJrjtxw" frameborder="0" allow="accelerometer; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
</center>

This **matrix of polynomials** now contains all our N impulse responses of our analysis filter bank, just like in our block transform case, but with **arbitrary long filters**. Unlike our transform case, here we now have z-transforms (polynomials in z) as our matrix entries. This now enables us to write the filtering and downsampling operations of our entire analysis filter bank as a simple multiplication using the above **polyphase matrix!** This is a very important result:

$$\large
Y(z)=X(z) \cdot H(z)$$

Mathematically this looks similar to the block transform case, but with z-transforms! Observe that this equation now contains **all the samples** of our **input signal** and also of our **subband signals** and our impulse responses, because we use the z-transformed signals. 

The z-transform converts a potentially **infinite sequence** into just a scalar or an element (its z-transform), which can be seen as a 1x1 matrix (the z-transform of that sequence is one big polynomial). All the input samples are in vector X(z). This is important because it allows us to use longer filters than just 1 block, longer than with the block transforms.

**Observe:** For an implementation, we can write the polyphase matrix as a polynomial of matrices H(m) ,


$$\large
H(z)=\sum_{m=0}^{\infty}H(m) \cdot z^{-m}$$

Where the elements of the n’th row and k’th column of **H(m)** are:

$$\large
H_\text{n,k}(z)=h_k(m \cdot N+n)$$

In Python the exponent *m* would appear as the third dimension, as “slices” of matrices,

$$\large
Hp[n,k,m] = h_k(mN+n)
$$

**In conclusion:** We can write the entire analysis filter bank with its N filters and downsamplers by N with a size NxN polyphase matrix H (z) , which is multiplied by the polyphase vector of the signal X (z) .

## Synthesis Filter Bank

In [3]:
%%html
<Center>
<iframe width="560" height="315" src="https://www.youtube.com/embed/XSZSsrVKZAs" frameborder="0" allow="accelerometer; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
</center>

Just as for the block transform case, we can also get a corresponding formulation for the synthesis filter bank. For the synthesis filter bank, we can now also write the reconstructed sequence $\hat x(n)$ in terms of blocks with index *m*
and phases *n*, and obtain the synthesis upsampling and convolution as (see also eq. (8) of lecture (11)):

$$\large
\hat x(mN+n) = \sum_{k=0}^{N-1} \sum_{m'=0}^{\frac{L}{N-1}} y_k(m-m') \cdot g_k(m'N+n)
$$
(with some still to be determined $g_k$)

We can now also use vectors for our sequences of blocks to simplify this equation, using a vector for our reconstructed signal and for our k'th synthesis filter (we start again with looking at just 1 filter),

$$\large
\hat x(m)=[\hat x(mN),\hat x(mN+1),...,\hat x(mN+N-1)]
$$

$$\large
g_k(m)=[g_k(mN),g_k(mN+1),...,g_k(mN+N-1)]
$$

Now we can re-write our synthesis equation as (with L the length of the synthesis filters):

$$\large
\hat x(m) = \sum_{k=0}^{N-1} \sum_{m'=0}^{\frac{L}{N-1}} y_k(m-m') \cdot g_k(m')
$$

The inner sum is a convolution,

$$\large
\hat x(m) = \sum_{k=0}^{N-1} y_k(m) * g_k(m)
$$

where we now no longer have our phase index *n*, because we now have output blocks instead of samples.

The inner sum is again a convolution, which turns into a multiplication using the z-transform,

$$\large
\hat X(z) = \sum_{k=0}^{N-1} Y_k(z)\cdot G_k(z)
$$

Now we can extend this notation to our bank of N synthesis filters using our subband vector Y(z), and the synthesis polyphase matrix.

Since the output is the sum of all subbands, we obtain our polyphase matrix by collecting all our polyphase (row) vectors of our synthesis filters $G_k(z)$ into a matrix, such that the **outer sum** of the above equation **turns into a matrix multiplication**,

$$\large
\hat X(z) = [Y_0(z), Y_1(z),...Y_{N-1}(z)] \cdot G(z)
$$

where each row of G( z) now contains one synthesis filter, or

$$\large
=\left[
\begin{array}{c}
G_0(z) \\
G_1(z) \\
\vdots \\
G_{N-1}(z)
\end{array} \right] 
=\left[ 
        \begin{array}{ccc} 
            G_{0,0}(z) & G_{0,1}(z) & \dots & G_{0,N-1}(z) \\ 
            G_{1,0}(z) & \dots & \dots & \vdots \\
            \vdots & \dots & \ddots & \vdots \\ 
            G_{N-1,0}(z) & \dots & \dots & G_{N-1,N-1}(z) 
        \end{array}
    \right]
$$

This is the **synthesis polyphase matrix.**

**Observe** that for this polyphase matrix, the indices for the subbands *k* and for the phase *n* are in reversed order compared to the analysis polyphase matrix.

**In Conclusion:** Again we turned the mathematically very complex operation of upsampling and synthesis filtering into a mathematically very simple operation with the multiplication of the subband vector with the polyphase matrix!

$$\large
\hat X(z) = Y(z) \cdot G(z)
$$

## Perfect Reconstruction

In [4]:
%%html
<center>
<iframe width="560" height="315" src="https://www.youtube.com/embed/YsizTX_9X4E" frameborder="0" allow="accelerometer; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
</center>

Perfect reconstruction (PR) is defned as a reconstructed signal which is identical to the original signal except for a delay,

$$\large
\hat x(n) = x(n-n_d)
$$

with some delay $n_d$ at the original sampling rate.

This delay usually results from our filtering and the downsampling and upsampling operations. To obtain PR we can simply take a look at the output of our synthesis filter bank,

$$\large
\hat X(z) = Y(z) \cdot G(z) = X(z) \cdot H(z) \cdot G(z)
$$

The structure of the polyphase analysis and synthesis filter bank can be seen also in the following picture,

<center>
    <img src="./images/msp_12_01_polyphase.jpg" width="800">
</center>

The structure with the delays and downsamplers on the left of the analysis polyphase matrix converts a sequence of samples at the high sampling rate into a sequence of blocks at the low sampling rate (“blocking”), which is what our Python function
“x2polyphase" is doing. 

Conversely, the structure to the right of the synthesis polyphase matrix converts a sequence of blocks at the low sampling rate into a sequence of samples at the high sampling rate (“deblocking”, with polyphase2x)

Here we can see that we obtain perfect reconstruction if we have the synthesis polyphase matrix as the inverse of the analysis polyphase times a delay by *d* blocks,

$$\large
G(z) = z^{-d} \cdot H^{-1}(z)
$$

(where *d* here is the delay at the downsampled rate.)

This is basically again like in our block transform case. This is now the constraint for obtaining PR. 

The question is, how do we obtain filters for PR? How do we invert a polyphase matrix, containing the polynomials? How do we get "good" synthesis filters?

A simple approach is analog to **orthogonal** block transform matrices, where the inverse is simply the transpose matrix. There the analysis and synthesis filters are identical, except for the time reversal for the analysis filters.

The corresponding property of polyphase matrices is called **"para-unitary"** (see also: Vaidyanathan: Multirate Systems and Filter Banks). It is defned as,

$$\large
\begin{equation}
H^{-1}(z) = H^T(z^{-1})
\end{equation}
$$

(The transposition is the case for real valued coefficients in the polyphase matrix. Otherwise we need conjugate transposition for the coeffients).

This definition is very similar to the definition for orthonormal matrices, except that we have the “ $z^{−1}$ “ on the right hand side (which corresponds to a time-reversal).

The advantage (or rather: one of its advantages) of a polyphase matrix with this property is, that we don't need to explicitly compute its inverse, we just need to transpose it and replace all z's by $z^{-1}$

**Observe**: if our polynomials in the polyphase matrix **only have zero'th order** (only a constant, no z, only $z^0=1$ ), then the polyphase matrix is identical to a transform matrix (for instance our DCT4). We obtain this simple case if our filters are no longer than N, they fit into one block.